# Homework 10: Cleaning data with Regular Expressions

Time to use regular expressions!

# Hints and notes

### Opening files in subdirectories

Notice that this notebook might be **homework/**, but!!! the csvs and text files might be in **homework/scraped/** or **/homework/scraped/minutes_pdfs** or **/homework/pdfs/**. To open a file in a subdirectory, instead of having the filename be `"file.csv"` you'll just use `"some/subfolder/file.csv"`

### Opening text files

This will open up a file, read it in and show you the first 500 characters.

```python
contents = open("your-filename.txt").read()
contents[0:500]
```

> You might need `open("your-filename.txt", encoding="utf8").read()`

### Using regex

For some dumb reason you need to put `r` in front of the string you use when you're talking about regex. Just plain `"(\d\d\d)"` will usually work, but *sometimes* it won't and you'll need `r"(\d\d\d)`. It's best to just use the `r` all of the time, if you can remember!

### Using `.str.extract`

When you use `.str.extract`, you're always going to **capture one thing** and save it to a new column. You need to wrap the things you're interested in with parenthesis `(` `)`.

```python
df['phone_number'] = df['old_column'].str.extract(r"My phone number is (\d\d\d-\d\d\d-\d\d\d\d)")
```

### Setting pandas options

Pandas has a lot of options, like how many columns or rows it will show you, or how many characters it will show in a column before it stops showing you anything. Here are a few useful ones:

* `display.max_cols`: Number of columns to show at once
* `display.max_rows`: Number of rows to show at once
* `display.max_colwidth`: Maximum number of characters displayed from a string

You can set them using `pd.set_option("display.max_rows", 1000)`, for example, to show 1000 rows at a time. You can find a lot more at https://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html

### Regular expressions reference

I personally think http://www.regular-expressions.info/ is a wonderful wonderful reference (and tutorial), even if it's ugly! But here's a quick reference for you:

* `\d` is a digit
* `\d*` is zero or more digits 
* `\d+` is one or more digits
* `.` matches anything for ONE character
* `.*` is "give me anything forever"
* `\s` is whitespace, a.k.a. spaces and tabs
* `\w` is a word character, which includes capital and lowercase letters, numbers and hyphens.
* You can put `*` after anything, so `\w*` would mean "as many word characters as you can find"
* `\b` is a word boundary (you'll need the `r""` thing for this one)
* `( )` is a "capture group" for saving something
* `\1` is used when doing find/replace to say "put the first captured group here" (note, it's a dollar sign instead of a backslash in some editors)
* `[ABCDE]` is a character class, which means "match one of these, I don't care which"
* dollar sign means "end of the line"
* caret ^ means "beginning of the line"
* `\.` means "no really seriously I mean a period not just anything"
* You can use `\` with anything else that would normally be a special character, too, not just periods. `(` or `[` or whatever.

### Cleaning up extracted columns

Sometimes you get `\n` (newlines) or spaces or `\t` (tabs) or stuff at the beginning or the end of your column. `.str.strip()` will usually take care of that, just attach it after your `.str.extract()`

After you extract something, it's still a string even though you look at it and know it's a number. Use `.astype(int)` to turn it into an integer (no decimal) or `.astype(float)` to turn it into a float (yes decimal)

### Writing regular expressions in general

Even if I'm using regex in pandas or Python, I like to test them in my text editor with "Find." The highlighting really helps me see if I'm matching things! I also like to think "what stays the same?" when designing patterns, write those parts first, then fill in the blanks with what I want to capture.

## Importing

There might be more, I just wanted to put this up here for the `pd.set_option` part. It allows you to see a lot of content in a single column of pandas, which will be important for some parts below.

In [1]:
import re
import pandas as pd
pd.set_option('display.max_colwidth', 500)

# Part 1: Using `.str.extract` to pull data from columns in pandas

## 1.1 H&M

Open up `hm.csv` from the `scraped` directory. I want **four new columns**:

1. `price_original`, the original price, one of the new price
2. `price_discounted`, the discounted price
3. `pct_discount`, the percent discount
4. `article_id`, the article id (from the url)

Save as **hm_cleaned.csv**.

**Note:** When you look at it, it... won't look right. I don't know why, pandas is weird. Look at the `price` column by itself using `df['price']` before you write your regex.

**Tip:** Remember that `$` is a special regex symbol! You might need to escape it.

**Tip:** When doing `.str.extract`, the whole match doesn't get captured, only what you put `()` around! Think about anchoring to different points of the string, or things in the string.

**Tip:** Not all prices have cents!

**Tip:** Your first instinct about how to compute the percent discount is probably wrong

In [2]:
hm = pd.read_csv("C:/Users/Weihua/Documents/Columbia_Summer/foundations/06-20-18/10-homework/scraped/hm.csv")

In [3]:
hm.head()

,name,price,url
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A


In [4]:
hm.shape

(60, 3)

In [5]:
# price_original, the original price, one of the new price

hm["price_original"] = hm["price"].str.extract(r" \$(\d*\.?\d*)").astype(float)

In [6]:
# price_discounted, the discounted price

hm["price_discounted"] = hm["price"].str.extract(r"\$(\d*?.?\d+) ").astype(float)

In [7]:
# pct_discount, the percent discount

hm["pct_discount"] = round((hm["price_original"] - hm["price_discounted"]) / hm["price_original"] * 100, 2)

In [8]:
# article_id, the article id (from the url)

hm["article_id"] = hm["url"].str.extract(r"article=(.*)$")

In [8]:
hm.head()
hm.shape

(60, 6)

In [9]:
hm.to_csv("hm_clean.csv", index = False)

In [11]:
# hm_clean = pd.read_csv("C:/Users/Weihua/Documents/Columbia_Summer/foundations/06-20-18/10-homework/hm_clean.csv")

# hm_clean.head()

## 1.2 Sci-Fi Authors

Open up `sci-fi.csv` to clean. Get rid of the `\n` on the title and and give me six new columns:

* `avg_rating`
* `rating_count`
* `total_score`
* `score_votes`
* `series` the series the book belongs to
* `series_no` the book in the series that it is

For series, I'm talking about e.g. `(The Hunger Games, #1)` is `series` "The Hunter Games" and `series_no` 1.

Save as **sci-fi_cleaned.csv**.

**Tip:** You don't need regex to clean the title - there's a special thing that removes whitespace from the beginning/end of strings

**Tip:** Remember that `(` and `)` are special characters

**BONUS:** When you make the `total_score` column, pay close attention to it. If you notice the problem, fix it.

**BONUS:** You don't need these columns to be numbers, but life would be better if they were. 

In [12]:
sci_fi = pd.read_csv("C:/Users/Weihua/Documents/Columbia_Summer/foundations/06-20-18/10-homework/scraped/sci-fi.csv")

In [13]:
sci_fi.head()

,full_rating,full_score,rank,title,url
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,\nThe Handmaid's Tale\n,/book/show/38447.The_Handmaid_s_Tale
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"\nThe Hunger Games (The Hunger Games, #1)\n",/book/show/2767052-the-hunger-games
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"\nFrankenstein, or The Modern Prometheus\n",/book/show/18490.Frankenstein_or_The_Modern_Prometheus
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"\nA Wrinkle in Time (A Wrinkle in Time Quintet, #1)\n",/book/show/18131.A_Wrinkle_in_Time
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,\nThe Left Hand of Darkness\n,/book/show/18423.The_Left_Hand_of_Darkness


In [14]:
sci_fi.shape

(100, 5)

In [15]:
sci_fi.dtypes

full_rating    object
full_score     object
rank            int64
title          object
url            object
dtype: object

In [16]:
#clean up title

sci_fi["title"] = sci_fi["title"].str.strip()

In [17]:
# avg_rating

sci_fi["avg_rating"] = sci_fi["full_rating"].str.extract(r"(\d.*) avg").astype(float)
#sci_fi["avg_rating"]

In [18]:
# rating_count

#select
sci_fi["rating_count"] = sci_fi["full_rating"].str.extract(r"— (.*) ratings")

#get rid of "," and turn it into int
sci_fi["rating_count"] = sci_fi["rating_count"].str.replace(",", "").astype("int")
#sci_fi["rating_count"]

In [19]:
# total_score
sci_fi["total_score"] = sci_fi["full_score"].str.extract(r"score: (.*),")

sci_fi["total_score"] = sci_fi["total_score"].str.replace(",", "").astype("int")
#sci_fi["total_score"]

In [20]:
# score_votes
sci_fi["score_votes"] = sci_fi["full_score"].str.extract(r"and\n(\d*) people voted").astype("int")
#sci_fi["score_votes"]

In [21]:
# series the series the book belongs to

sci_fi["series"] = sci_fi["title"].str.extract(r"\((.*),?#")
sci_fi["series"] = sci_fi["series"].str.replace(",", "")
#sci_fi["series"]

In [22]:
# series_no the book in the series that it is
sci_fi["series_no"] = sci_fi["title"].str.extract(r"\(.*#(.*)\)")
#sci_fi["series_no"]

In [23]:
sci_fi.to_csv("sci_fi_clean.csv", index = False)

# sci_fi_clean = pd.read_csv("C:/Users/Weihua/Documents/Columbia_Summer/foundations/06-20-18/10-homework/sci_fi_clean.csv")
# sci_fi_clean.head()

## 1.3 Where you're just doing one of my former students' projects

Once upon a time my student Stefan did a project that involved some lawyer stuff. Most of the content was in PDFs, though! I converted them to text files and put them into the `pdfs` folder, and gave you code below to open up each of them and save their contents into a dataframe.

What a nice dataframe! I want you to add the following columns to it:

* `lawyer_app`, the applicant's lawyer (pro se means that they did it themselves, that's fine)
* `lawyer_gov`, the government's lawyer
* `judge`, the name of the judge
* `access`, whether the clearance is granted or denied (although you might miss a few)

Save as **court_cleaned.csv**.

**Note:** You can look at the original PDFs, they're also included.

**Note:** This uses a fun utility called `glob`, which is mostly fun because you use it as `glob.glob`. It's used to find files that match a certain filename pattern.

**BONUS:** You'll be happy once you get the judge, but make sure it doesn't have any extra punctuation on it.

**BONUS:** You can for some words using `.str.contains("blah")` and save it into new columns. Maybe `has_debt`, `has_bankruptcy`, etc.

> It's okay if it isn't perfect. Converting PDF into data rarely is! Usually you get 90% of it done with computers, then send people to enter the other 10% by hand.

In [24]:
import glob
filenames = glob.glob("C:/Users/Weihua/Documents/Columbia_Summer/foundations/06-20-18/10-homework-pdfs/pdfs/*.txt")
contents = [open(filename, encoding="utf8").read() for filename in filenames]
df = pd.DataFrame({'filename': filenames, 'content': contents})

In [25]:
df.head()

,filename,content
0,C:/Users/Weihua/Documents/Columbia_Summer/foundations/06-20-18/10-homework-pdfs/pdfs\11-02438.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-02438 \n\nFor Government: Stephanie C. Hess, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\..."
1,C:/Users/Weihua/Documents/Columbia_Summer/foundations/06-20-18/10-homework-pdfs/pdfs\11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 11-03073 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\nFor Government: Robert J. Kilmartin, Esq., Department Counse..."
2,C:/Users/Weihua/Documents/Columbia_Summer/foundations/06-20-18/10-homework-pdfs/pdfs\11-04909.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-04909 \n\n \n\nFor Government: Richard Stevens, Esq., Department Counsel \n\nFor Applicant: Pro se \n\n \n\n \nDUFFY, James F., Administrative Judge: \n\n \nApplicant mitigated \n\nconsiderations). Clearance is ..."
3,C:/Users/Weihua/Documents/Columbia_Summer/foundations/06-20-18/10-homework-pdfs/pdfs\11-07728.h1.pdf.txt,"DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n------------------------ \n \n\n \n\nISCR Case No. 11-07728 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n\n \n \n\nAppearances \n\n___________ \n\n \nDecision \n\n___________ \n\nFor Government: Julie R. Mendez, Esq., Department Counsel \n\nFor Applicant: Mark S. Zaid, Esq. \n\n \n\nHARVEY, Mark, Administrative Judge: \n \..."
4,C:/Users/Weihua/Documents/Columbia_Summer/foundations/06-20-18/10-homework-pdfs/pdfs\11-08313.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n\nISCR Case No. 11-08313 \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n--------------- \n \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\nFor Government: Julie R. Mendez, Esquire, Department Counsel \n\nFo..."


Okay, now do the work and **make those new columns!**

In [26]:
df.shape

(15, 2)

In [27]:
df["lawyer_app"] = df["content"].str.extract(r"For Applicant\s{0,}(.*)", re.IGNORECASE)

df["lawyer_app"] = df["lawyer_app"].str.replace(":","")
df["lawyer_app"] = df["lawyer_app"].str.strip()

#df["lawyer_app"]

In [28]:
df["lawyer_gov"] = df["content"].str.extract(r"For Government\s{0,}(.*)", re.IGNORECASE)

df["lawyer_gov"] = df["lawyer_gov"].str.replace(":","")
df["lawyer_gov"] = df["lawyer_gov"].str.strip()

#df["lawyer_gov"]

In [49]:
# judge, the name of the judge

df["judge"] = df["content"].str.extract(r"(\w.*)\n{1,}Administrative Judge")

#df["judge"]

In [39]:
# access, whether the clearance is granted or denied (although you might miss a few)
df["access"] = df["content"].str.extract(r"is ([gad].*ed)\.",re.IGNORECASE)
df["access"] = df["access"].str.replace("AFFIRMED", "affirmed")
#is there a difference between afrirmed and granted ...?

In [42]:
df["has_foreign"] = df["content"].str.contains("foreign", re.IGNORECASE)
df["has_China"] = df["content"].str.contains("China", re.IGNORECASE)

In [48]:
df.to_csv("court_cleaned.csv", index = False)

# court_cleaned = pd.read_csv("C:/Users/Weihua/Documents/Columbia_Summer/foundations/06-20-18/10-homework/court_cleaned.csv")
# court_cleaned.head()

# Reading books

When you're doing text work, you're legally obligated work on Jane Austen's Pride and Prejudice (at least I *think* so). Let's do some naive analysis of it!

## Read in Jane Austen's Pride and Prejudice (without moving the file!)

It's in the `data/` directory, and named `Austen_Pride.txt`.

In [10]:
pp = open("C:/Users/Weihua/Documents/Columbia_Summer/foundations/06-20-18/10-homework/data/Austen_Pride.txt", encoding="utf8").read()

## Look at the first 500 or so characters of it 

In [11]:
pp[0:500]

' Pride and Prejudice\nby Jane Austen\nChapter 1\nIt is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.\nHowever little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.\n"My dear Mr. Bennet," said his lady to him one day, "have you heard that Nethe'

## Use a regular expression to find every "he" or "she" in the book. There should be about 3000 of them.

**Tip:** Do you know about **word boundaries?** `\b` means "the beginning of end of a word."

**Tip:** You might also want to use `re.IGNORECASE`. Maybe you'll need to google it? 

**Tip:** Do NOT use `re.compile`

In [24]:
my_regex = r"[^a-z](s?he\b)"
hshe = (re.findall(my_regex, pp, re.IGNORECASE))
hshe
#len(hshe)

['he',
 'he',
 'she',
 'she',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'she',
 'he',
 'he',
 'she',
 'he',
 'She',
 'she',
 'she',
 'He',
 'he',
 'she',
 'he',
 'She',
 'She',
 'she',
 'she',
 'she',
 'he',
 'he',
 'he',
 'she',
 'she',
 'he',
 'he',
 'she',
 'he',
 'he',
 'He',
 'he',
 'He',
 'he',
 'he',
 'he',
 'She',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'He',
 'he',
 'he',
 'She',
 'he',
 'he',
 'She',
 'She',
 'she',
 'she',
 'he',
 'he',
 'He',
 'he',
 'he',
 'she',
 'she',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'she',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'He',
 'she',
 'She',
 'she',
 'he',
 'He',
 'he',
 'she',
 'He',
 'she',
 'He',
 'he',
 'He',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'He',
 'he',
 'He',
 'he',
 'he',
 'he',
 'he',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'she',
 'he',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he

## Use a regular expression to find those same "he" or "she"s, but also match *the word after it*

The first four should be:

* he is
* he had
* she told
* he came

In [26]:
my_regex = r"[^a-z](s?he\s\w+)"
hshew = (re.findall(my_regex, pp, re.IGNORECASE))
hshew[:4]
#len(hshew)

['he is', 'he had', 'she told', 'he came']

## Use capture groups to save the pronoun (he/she) as one match and the word as another

The first five should look like

```
[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed')]```

In [31]:
my_regex = r"[^a-z](s?he)\s(\w+)"
hshewg = (re.findall(my_regex, pp, re.IGNORECASE))
hshewg[:5]

[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed')]

## Save those matches into a dataframe

You can give the column names with `columns=['pronoun', 'verb']`

In [34]:
columns = []

for word in hshewg:
    word_dict = {}
    word_dict["pronoun"] = word[0]
    word_dict["verb"] = word[1]
    columns.append(word_dict)

columns[:5]

[{'pronoun': 'he', 'verb': 'is'},
 {'pronoun': 'he', 'verb': 'had'},
 {'pronoun': 'she', 'verb': 'told'},
 {'pronoun': 'he', 'verb': 'came'},
 {'pronoun': 'he', 'verb': 'agreed'}]

## How many times is each pronoun used?

In [49]:
import pandas as pd

df = pd.DataFrame(columns)
df.head()

,pronoun,verb
0,he,is
1,he,had
2,she,told
3,he,came
4,he,agreed


In [51]:
df.pronoun.value_counts()

she    1323
he     1054
She     325
He      234
Name: pronoun, dtype: int64

## Oh, wait, clean that up.

Make it only 'he' and 'she' lowercase.

It should be about 1600 'she' and 1300 'he'

In [52]:
df["pronoun"] = df["pronoun"].str.lower()

In [53]:
df.pronoun.value_counts()

she    1648
he     1288
Name: pronoun, dtype: int64

## What are the top 20 most common verbs?

In [59]:
df.verb.value_counts().head(20)

was          372
had          371
could        172
is           139
would         94
has           70
did           67
will          50
might         46
should        41
felt          38
must          37
said          33
saw           32
added         31
thought       31
then          26
replied       22
looked        21
continued     21
Name: verb, dtype: int64

## What are the top 20 most common verbs for 'he', and the top 20 most common for 'she'

**Tip:** Don't use groupby, just filter. If you want to know how, though, you can also look at "value counts for different categories" on [this page](http://jonathansoma.com/lede/foundations-2017/classes/more-pandas/class-notes/)

In [64]:
df[df.pronoun == "she"].verb.value_counts().head(20)

was        212
had        205
could      132
is          65
would       59
did         38
felt        33
saw         29
will        26
might       25
added       23
has         21
said        20
thought     18
then        15
should      15
found       13
must        13
soon        12
cried       11
Name: verb, dtype: int64

In [65]:
df[df.pronoun == "he"].verb.value_counts().head(20)

had        166
was        160
is          74
has         49
could       40
would       35
did         29
should      26
must        24
will        24
might       21
replied     14
may         13
came        13
thought     13
said        13
does        12
never       12
meant       11
then        11
Name: verb, dtype: int64

## Who cries more, men or women? Give me a percentage answer.

**Tip:** It's `cried`, because of, you know, how books are written

In [69]:
df[df.verb == "cried"].pronoun.value_counts(normalize=True)

she    0.916667
he     0.083333
Name: pronoun, dtype: float64

## How much more common is 'he' than 'she' in J.R.R. Tolkein's Fellowship of the Ring? How does that compare to Pride and Prejudice?

The book is in the same directory.

In [72]:
lr = open("C:/Users/Weihua/Documents/Columbia_Summer/foundations/06-20-18/10-homework/data/Lord of the Rings - 01 - The Fellowship of the Ring - J. R. R. Tolkien - 1955.txt", encoding="utf8").read()

In [74]:
lr[:500]

'THE FELLOWSHIP OF THE RING\n\n\n\n\nBEING THE FIRST PART OF\n\nTHE LORD OF THE RINGS\n\nBY\n\nJ.R.R. TOLKIEN\n\n\n\n\n\nThree Rings for the Elven-kings under the sky,\n\nSeven for the Dwarf-lords in their halls of stone,\n\nNine for Mortal Men doomed to die,\n\nOne for the Dark Lord on his dark throne\n\nIn the Land of Mordor where the Shadows lie.\n\nOne Ring to rule them all, One Ring to find them,\n\nOne Ring to bring them all and in the darkness bind them\n\nIn the Land of Mordor where the Shadows lie.\n\n\n\n\n\nCONTENTS\n\n\nCOV'

In [76]:
my_regex = r"[^a-z](s?he)\s(\w+)"
pv = (re.findall(my_regex, lr, re.IGNORECASE))
pv[:5]

[('he', 'had'),
 ('He', 'was'),
 ('he', 'wrote'),
 ('he', 'recorded'),
 ('he', 'had')]

In [77]:
columns_lr = []

for word in pv:
    word_dict = {}
    word_dict["pronoun"] = word[0]
    word_dict["verb"] = word[1]
    columns_lr.append(word_dict)

columns_lr[:5]

[{'pronoun': 'he', 'verb': 'had'},
 {'pronoun': 'He', 'verb': 'was'},
 {'pronoun': 'he', 'verb': 'wrote'},
 {'pronoun': 'he', 'verb': 'recorded'},
 {'pronoun': 'he', 'verb': 'had'}]

In [79]:
df_lr = pd.DataFrame(columns_lr)
#df_lr.head()
df_lr["pronoun"] = df_lr["pronoun"].str.lower()

In [80]:
df[df.verb == "cried"].pronoun.value_counts(normalize=True)

she    0.916667
he     0.083333
Name: pronoun, dtype: float64

In [84]:
df_lr[df_lr.verb == "cried"].pronoun.value_counts(normalize=True)
#guys in lord of the right cries a lot, that's for sure lol

he     0.978261
she    0.021739
Name: pronoun, dtype: float64

In [103]:
df.pronoun.value_counts()

she    1648
he     1288
Name: pronoun, dtype: int64

In [104]:
df_lr.pronoun.value_counts()
#but then again, there are WAY more man than woman in lord of the ring.

he     2986
she     156
Name: pronoun, dtype: int64